In [2]:
import torch
import torch.nn as nn

import sys
sys.path.append('..')
from d2l_helpers import *
import time

In [3]:
def vgg_block(num_convs, in_channels, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers)

In [4]:
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))

def vgg(conv_arch):
    conv_blks = []
    in_channels = 1
    for (num_convs, out_channels) in conv_arch:
        conv_blks.append(vgg_block(num_convs, in_channels, out_channels))
        in_channels = out_channels
    return nn.Sequential(
        *conv_blks, nn.Flatten(),
        nn.Linear(out_channels * 7 * 7, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 10)
    )

net = vgg(conv_arch)

In [5]:
X = torch.randn(size=(1, 1, 224, 224))
for blk in net:
    X = blk(X)
    print(blk.__class__.__name__, 'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 64, 112, 112])
Sequential output shape:	 torch.Size([1, 128, 56, 56])
Sequential output shape:	 torch.Size([1, 256, 28, 28])
Sequential output shape:	 torch.Size([1, 512, 14, 14])
Sequential output shape:	 torch.Size([1, 512, 7, 7])
Flatten output shape:	 torch.Size([1, 25088])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 10])


In [8]:
ratio = 4
small_conv_arch = [(pair[0], pair[1] // ratio) for pair in conv_arch]
net = vgg(small_conv_arch)

device = get_device()

In [9]:
lr, num_epochs, batch_size = 0.05, 10, 128
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=224)
train_cnn(net, train_iter, test_iter, num_epochs, lr, device)

training on mps
epoch 1, iter 93/469
epoch 1, iter 186/469
epoch 1, iter 279/469
epoch 1, iter 372/469
epoch 1, iter 465/469
epoch 1, iter 469/469
loss 1.348, train acc 0.511, test acc 0.814
epoch 2, iter 93/469
epoch 2, iter 186/469
epoch 2, iter 279/469
epoch 2, iter 372/469
epoch 2, iter 465/469
epoch 2, iter 469/469
loss 0.440, train acc 0.840, test acc 0.864
epoch 3, iter 93/469
epoch 3, iter 186/469
epoch 3, iter 279/469
epoch 3, iter 372/469
epoch 3, iter 465/469
epoch 3, iter 469/469
loss 0.350, train acc 0.873, test acc 0.880
epoch 4, iter 93/469
epoch 4, iter 186/469
epoch 4, iter 279/469
epoch 4, iter 372/469
epoch 4, iter 465/469
epoch 4, iter 469/469
loss 0.302, train acc 0.889, test acc 0.896
epoch 5, iter 93/469
epoch 5, iter 186/469
epoch 5, iter 279/469
epoch 5, iter 372/469
epoch 5, iter 465/469
epoch 5, iter 469/469
loss 0.271, train acc 0.901, test acc 0.897
epoch 6, iter 93/469
epoch 6, iter 186/469
epoch 6, iter 279/469
epoch 6, iter 372/469
epoch 6, iter 465/469
